In [322]:
import pandas as pd
from collections import defaultdict
from dataclasses import dataclass, field
from glob import glob
import numpy as np
import math
import datetime as dt
from functools import partial, reduce
from typing import Callable
import pandas as pd
import pysam
from operator import methodcaller
from abc import ABC, abstractmethod
from collections import Counter
import re
import scipy.stats as stats


In [323]:

class Processing(ABC):
    '''Operations on pileup df'''
    @abstractmethod
    def operation():
        pass

class RemoveRedundantColumns(Processing):
    '''Compute Max'''
    
    def operation(df: pd.DataFrame) -> pd.DataFrame:
        for col in ['ref', 'seq']:
            if not df[f'{col}_n'].equals(df[f'{col}_t']):
                raise ValueError(f'{col} differs!')
            df[col] = df[f'{col}_n']
            del df[f'{col}_n']
            del df[f'{col}_t']
   
        return df

class RemoveRedundantColumns(Processing):
    '''Compute Max'''
    
    def operation(df: pd.DataFrame) -> pd.DataFrame:
        for col in ['ref', 'seq']:
            if not df[f'{col}_n'].equals(df[f'{col}_t']):
                raise ValueError(f'{col} differs!')
            df[col] = df[f'{col}_n']
            del df[f'{col}_n']
            del df[f'{col}_t']
   
        return df


def process_pile(df):
    # __subclasses__ will found all classes inheriting from Operations
    for step in Processing.__subclasses__():
        df = step.operation(df)

In [324]:
# def separate_indels_from_res(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
#     def find_indels_in_res(res):
#         def extract_indel(res, val):
#             indels = []
#             bits = res.strip().split(val)
#             if len(bits) > 1:
#                 res = ''
#                 for bit in bits:
#                     if bit[0].isdigit():
#                         indel_len = []
#                         for char in bit:
#                             if char.isdigit():
#                                 indel_len.append(char)
#                             else:
#                                 break
#                         indel_len = ''.join(indel_len)
#                         indel = bit[len(indel_len):int(indel_len) + len(indel_len)]
#                         indels.append(val + ':' + indel)
#                         res += bit[len(indel_len) + int(indel_len):]
#                     else:
#                         res += bit
#             return indels, res
#         ins, res = extract_indel(res, '+')
#         dels, res = extract_indel(res, '-')
#         indels = ins+dels
#         res = ''.join(char for char in res if char in '.,ATCGatcg*')
#         most_seen_indel = Counter(indels).most_common()[0] if indels else 'NA'
#         return pd.Series([res, most_seen_indel])
#     df[[f'{sample_type}_indel_free_res', f'{sample_type}_indels']] = df[f'res_{sample_type}_no_carrot'].apply(find_indels_in_res)
#     df[f'{sample_type}l1'] = df[f'{sample_type}_indel_free_res'].str.len() 
#     df[f'{sample_type}l2'] = df[f'qual_{sample_type}'].str.len()
#     if not df[f'{sample_type}l1'].equals(df[f'{sample_type}l2']):
#       raise ValueError(f'{sample_type}_indel_free_res and qual_{sample_type} should have same length!! {df[f"{sample_type}l1"].compare(df[f"{sample_type}l1"])}')
    
# #     return df

# ref_up = tmp_d.get('ref').upper()
# to_keep = ['.', ',', 'A', 'T', 'C', 'G', '*']  # keep * cuz have phred score...
# if i == 0:
#     res = ''.join([nuc for nuc in res if nuc.upper() in to_keep])
#     res = res.replace('.', ref_up).replace(',', tmp_d.get('ref').lower())
# else:
#     res = ''.join([nuc for nuc in res if nuc.upper() in to_keep])
#     res = res.replace('.', ref_up).replace(',', ref_up)
# if not len(res) == len(tmp_d.get('qual')):
#     print('not_same_len', len(res), len(tmp_d.get('qual')),
#             res, tmp_d, i)

NameError: name 'tmp_d' is not defined

In [328]:
import operator


Preprocessor = Callable[[pd.DataFrame, str], pd.DataFrame]

class DataSchema:
    AMOUNT = "amount"
    CATEGORY = "category"
    DATE = "date"
    MONTH = "month"
    YEAR = "year"

class PileSchema:
    AMOUNT = "amount"
    CATEGORY = "category"
    DATE = "date"
    MONTH = "month"
    YEAR = "year"

def create_df(pile):#: str, chrom: str, start: int, end: int) -> pd.DataFrame:
    '''Converts a genomic region of a pileup to a df'''
    keys = ['seq', 'pos', 'ref', 'reads', 'res', 'qual', 'pos_in_read', 'read_names', 'flags', 'map']#TODO drop 'pos_in_read', 'read_names',
    tabixfile = pysam.TabixFile(pile)
    df = pd.DataFrame([line.split('\t') for line in tabixfile.fetch()], columns=keys)
    df.set_index('pos', inplace=True)
    return df

def remove_redundant_column(df: pd.DataFrame, col_name: str) -> pd.DataFrame:

    if not df[f'{col_name}_n'].equals(df[f'{col_name}_t']):
        raise ValueError(f'{col_name} differs!')
    df[col_name] = df[f'{col_name}_n']
    del df[f'{col_name}_n']
    del df[f'{col_name}_t']
   
    return df

def create_SV_column(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
    df[f'SV_{sample_type}'] = df[f'res_{sample_type}'].str.count('^') + df[f'res_{sample_type}'].str.count('$')
    return df

def create_indel_column(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
    df[f'SV_{sample_type}'] = df[f'res_{sample_type}'].str.count('^') + df[f'res_{sample_type}'].str.count('$')
    return df

def remove_carrots_from_res(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
    # ^ (caret) marks the start of a read segment and the ASCII
    # of the character following `^' minus 33 gives the mapping quality
    def slice_and_dice(res):
        if '^' in res:
            if res.startswith('^'):
                res = ''.join([bit[1:] for bit in res.split('^')])
            else:
                res = ''.join([bit[1:] if j != 0 else bit for j, bit in enumerate(res.split('^'))])
        return res
    df[f'res_{sample_type}_no_carrot'] = df[f'res_{sample_type}'].apply(slice_and_dice)
    return df

#def remove_low_quality(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
def convert_to_upper(res, ref):
    assert ref.isupper()
    return res.replace(',', ref).replace('.', ref).replace('*', '').upper()

def remove_ref_from_tumour_res(res_n, res_t, ref):

    most_common_base_in_normal = Counter(convert_to_upper(res_n, ref)).most_common()[0][0]
    return convert_to_upper(res_t, ref).replace(most_common_base_in_normal, '')

def remove_positions_with_little_support_for_somatic_var(df: pd.DataFrame) -> pd.DataFrame:
    def putative_vars(res_t, res_n, ref):
        # res_t = convert_to_upper(res_t, ref)
        # res_n = convert_to_upper(res_n, ref)
        # most_common_base_in_normal = Counter(res_n).most_common()[0][0]
        res_t = remove_ref_from_tumour_res(res_n, res_t, ref)
        if res_t:
            return True if Counter(res_t).most_common()[0][1] > 4 else False
        return False
    return df[df.apply(lambda x: putative_vars(
        x['res_t_no_carrot'],
        x['res_n_no_carrot'],
        x['ref']),
        axis = 1
    )]
    
#tried complied regex, still slow
def separate_indels_from_res(df: pd.DataFrame, sample_type: str) -> pd.DataFrame:
    def find_indels_in_res(res):
        regex_result = re.findall("\+[0-9]+[ACGTNacgtn]+|\-[0-9]+[ACGTNacgtn]+", res)
        corected_regex_result = []#can't find other way to stop regex adding unrelated nucs at end
        for indel in regex_result:
            indel_len = ''.join(char for char in indel if char.isdigit())
            corected_regex_result.append(indel[0] + indel_len + ''.join(char for char in indel if char.isalpha())[:int(indel_len)])
        indel_count = Counter(corected_regex_result)
        for indel in indel_count.keys():
            res = ''.join(res.split(indel))
        most_seen_indel = indel_count.most_common()[0] if indel_count else 'NA'
        most_seen_indel_upper = np.nan if most_seen_indel == 'NA' else (most_seen_indel[0].upper(), most_seen_indel[1])
        res = ''.join(char for char in res if char in '.,ATCGatcg*')
        return pd.Series([res, most_seen_indel_upper])
    df[[f'{sample_type}_indel_free_res', f'{sample_type}_indels']] = df[f'res_{sample_type}_no_carrot'].apply(find_indels_in_res)
    if not df[f'{sample_type}_indel_free_res'].str.len().equals(df[f'qual_{sample_type}'].str.len()):
      raise ValueError(f'{sample_type}_indel_free_res and qual_{sample_type} should have same length!! {df[f"{sample_type}l2"].compare(df[f"{sample_type}l1"])}')
    return df

def caller(count, normal_count, reads_t, reads_n, putative_somatic_var, vars):

    vals = [count, normal_count]
    nons = [int(reads_t) - count, int(reads_n) - normal_count]
    oddsratio, pvalue1 = stats.fisher_exact([vals, nons], alternative='greater')
    if pvalue1 < 0.01:
        vars.append((putative_somatic_var, pvalue1))
    return vars
    
def call_var(df: pd.DataFrame) -> pd.DataFrame:
    def test_putative_somatic_SNVs(tumour_nucs, normal_nucs, reads_t, reads_n, ref, res_n):
        normal_nucs_upper = convert_to_upper(normal_nucs, ref)
        tumour_nucs_no_ref_upper = remove_ref_from_tumour_res(normal_nucs, tumour_nucs, ref)
        normal_nucs_most_common = dict(Counter(normal_nucs_upper).most_common())
        tumour_nucs_most_common = {
            t_var: count for t_var, count in Counter(tumour_nucs_no_ref_upper).items() if normal_nucs_most_common.get(t_var, 0) < 2
        }
        vars = []
        for putative_somatic_var, count in tumour_nucs_most_common.items():
            if count > 4:
                normal_count = normal_nucs_most_common.get(putative_somatic_var, 0)
                vars = caller(count, normal_count, reads_t, reads_n, putative_somatic_var, vars)
        return vars if vars else np.nan
    df["SNV_calls"] = df.apply(lambda x: test_putative_somatic_SNVs(
        x['t_indel_free_res'],
        x['n_indel_free_res'],
        x['reads_t'],
        x['reads_n'],
        x['ref'],
        x['res_n']),
        axis = 1
    )
    def test_top_putative_somatic_indel(tumour_indels, reads_t, reads_n, ref, res_n):
        vars = []
        if type(tumour_indels) != float:
            freq_of_most_common_tumour_indel_in_normal = res_n.upper().count(tumour_indels[0])
            if freq_of_most_common_tumour_indel_in_normal < 2:
                normal_d = dict([(tumour_indels[0], freq_of_most_common_tumour_indel_in_normal)])
                putative_somatic_var, count = tumour_indels
                if count > 4:
                    normal_count = normal_d.get(putative_somatic_var, 0)
                    vars = caller(count, normal_count, reads_t, reads_n, putative_somatic_var, vars)
        return vars if vars else np.nan
    df["indel_call"] = df.apply(lambda x: test_top_putative_somatic_indel(
        x['t_indels'],
        x['reads_t'],
        x['reads_n'],
        x['ref'],
        x['res_n']),
        axis = 1
    )
    def test_top_putative_somatic_SV(tumour_SVs, normal_SVs, reads_t, reads_n, ref, res_n):
        #no limit as some reads naturally start and stop....
        vars = []
        if tumour_SVs > 4:
            vars = caller(tumour_SVs, normal_SVs, reads_t, reads_n, 'SV', vars)
        return vars if vars else np.nan
    df["SV_call"] = df.apply(lambda x: test_top_putative_somatic_SV(
        x['SV_t'],
        x['SV_n'],
        x['reads_t'],
        x['reads_n'],
        x['ref'],
        x['res_n']),
        axis = 1
    )
    return df

In [333]:
df_tumour = create_df('/Users/liam/code/personal/SBSC/tests/resources/HCC1937_t_RNF157.pileup.gz')
df_normal = create_df('/Users/liam/code/personal/SBSC/tests/resources/HCC1937_n_RNF157.pileup.gz')
df = df_normal.join(df_tumour, how='inner', lsuffix='_n', rsuffix='_t')
print(df.shape)
df = remove_redundant_column(df, 'ref')
df = remove_redundant_column(df, 'seq')
df = create_SV_column(df, 't')
df = create_SV_column(df, 'n')
df = remove_carrots_from_res(df, 't')
df = remove_carrots_from_res(df, 'n')
df = remove_positions_with_little_support_for_somatic_var(df)
print(df.shape)
df = separate_indels_from_res(df, 't')
df = separate_indels_from_res(df, 'n')
df = call_var(df)
df = df[df[['SNV_calls', 'indel_call', 'SV_call']].notna().any(1)]
df.shape

(98038, 18)
(20488, 20)


(57, 27)

In [383]:
@dataclass 
class PileupSchema:#TODO ???drop 'pos_in_read', 'read_names'????
    SEQUENCE = "seq"
    POSITION = "pos"
    REFERENCE = "ref"
    READS = "reads"
    RESULTS = "res"
    QUALITY = "qual"
    POSITION_IN_READ = "pos_in_read"
    FLAGS = "flags"
    MAPPING_QUALITY = "map"

@dataclass
class ProcessedSchema(PileupSchema):
    STRUCTURAL_VARIANT = "SV"

@dataclass
class TumourSchema(ProcessedSchema):
    
    @property
    def SV():
        return f'{TumourSchema.STRUCTURAL_VARIANT}_tumour'
t=TumourSchema()
t.SV

TypeError: TumourSchema.SV() takes 0 positional arguments but 1 was given

In [338]:
idx = df.ref.index.get_loc('76145432')

df.ref.iloc[idx - 10 : idx + 10]

Series([], Name: ref, dtype: object)

In [349]:
class DataSchema:
    AMOUNT = "amount"
    CATEGORY = "category"
    DATE = "date"
    MONTH = "month"
    YEAR = "year"

   

def create_df(pile):#: str, chrom: str, start: int, end: int) -> pd.DataFrame:
    '''Converts a genomic region of a pileup to a df'''
    keys = ['seq', 'pos', 'ref', 'reads', 'res', 'qual', 'pos_in_read', 'read_names', 'flags', 'map']#TODO drop 'pos_in_read', 'read_names',
    


In [351]:
a= [name for schema, name in vars(DataSchema).items() if not schema.startswith("__")]
a

['amount', 'category', 'date', 'month', 'year']

In [377]:

class PileupSchema:#TODO ???drop 'pos_in_read', 'read_names'????
    SEQUENCE = "seq"
    POSITION = "pos"
    REFERENCE = "ref"
    READS = "reads"
    RESULTS = "res"
    QUALITY = "qual"
    POSITION_IN_READ = "pos_in_read"
    FLAGS = "flags"
    MAPPING_QUALITY = "map"


class ProcessedDataSchema(PileupSchema):
    AMOUNT = "amount"
    CATEGORY = "category"
    DATE = "date"
    MONTH = "month"
    YEAR = "year"

    
def ff(sample_type, name: ProcessedDataSchema):
    return f'{sample_type}_{name}'

tumour_cols = partial(ff, 'tumour')
normal_cols = partial(ff, 'normal')

tumour_cols(ProcessedDataSchema.AMOUNT), normal_cols(ProcessedDataSchema.POSITION_IN_READ)


('tumour_amount', 'normal_pos_in_read')

In [372]:
p=PileupSchema()
print(PileupSchema.list_all_columns())

['seq', 'pos', 'ref', 'reads', 'res', 'qual', 'pos_in_read', 'flags', 'map']


In [321]:
df[df[['SNV_calls', 'indel_call', 'SV_call']].notna().any(1)][['res_t_no_carrot','res_n_no_carrot', 'SNV_calls', 'indel_call', 'SV_call', 't_indels']].to_csv('~/Downloads/regex.csv')

In [260]:
df.loc['76157884',:]



seq_n                                                        chr17
ref_n                                                            C
reads_n                                                         59
res_n            *A,,*.,,,.,,.*,*,,.,,.*,,.**,,,,.,*tAA,,,t,.*A...
qual_n           ('8/)9=9&>848/4287@5B:&&36%-).;82-/47(941';8%4...
pos_in_read_n    14876,13621,13831,13241,9962,9469,8150,7546,76...
read_names_n     a7f99129-4d5a-42b1-994b-4e4fcce73ce1,5e17a39a-...
flags_n          0,0,16,16,16,0,16,16,16,0,16,16,0,0,16,0,16,16...
map_n            WY[\XY[ZY]YZWNX]]]Z\[ZXY[ZRW[WS]YT]SZ]]X]]]Z[]...
seq_t                                                        chr17
ref_t                                                            C
reads_t                                                         51
res_t            *AGt,,,a.A.,,AA,,A.A,*,..,,*A,A,A,,A,.A*,,,aAa...
qual_t           '5%$<4766(+8<1.;57007&1%8273-//3:-46,74868+>-?...
pos_in_read_t    15568,10657,9258,7684,7560,7148,7008,6871,700

In [58]:

#failed attempt for do regex faster
chars, index = [], []
for i, char in enumerate(a):
    if char in '+-':
        chars.append('|')
        index.append(i)
    if char.isdigit():
        chars.append(char)
        index.append(i)

kk = ''.join(chars)
index = [pos for i, pos in enumerate(index) if kk[i] == '|']
assert len(kk.split('|')[1:]) == len(index)
indels = kk.split('|')[1:]
for i, indel in zip(index, indels):
    print(a[(i): i+len(indel)+int(indel)])


-1
+1
+1
-2t
-2t
-11tttttttttt
-1


In [ ]:
,,,.*.,,-1t,.,.....,,..-2TT.,-1t,.,.-1T.-1T,,....	..-2TT,,*....+2TT.-2TT,-1t,-1t....,,+1t.*...+1...	,,,.*.,,,.,.....,,...,,.,..,,.,..,..,.,,,..,,....	NA	44	44	..,,*.....,,....,,.*......,.,.,.,,,,,.,,....

In [126]:
re.findall("\+[0-9]+[ACGTNacgtn]+|\-[0-9]+[ACGTNacgtn]+", ',,,.*.,,-1t,.,.....,,..-2TT.,-1t,.,.-1T.-1T,,....')

['-1t', '-2TT', '-1t', '-1T', '-1T']

In [60]:
#wtf!! .-1C.,,.-1C...-1C.,,....-1C,,.......-1C.,.,.,+...    @76142572 why + by itself?? ... is truncation... ffs
df[df.res_n.str.contains('$', regex=False)]#.to_csv('~/Downloads/dola.csv')
df[df.res_n.str.contains('^', regex=False)]#.to_csv('~/Downloads/carrot.csv')

,reads_n,res_n,qual_n,pos_in_read_n,read_names_n,flags_n,map_n,reads_t,res_t,qual_t,...,SV_t,SV_n,res_n_no_carrot,res_t_no_carrot,t_indel_free_res,t_ins,t_dels,n_indel_free_res,n_ins,n_dels
pos,,,,,,,,,,,,,,,,,,,,,
76142500,45,"..*,......,,....,,........,.,.,.,,,,,.,,....^S,",9F#I%F3D6B>I1C>B-IDE7=AFD<>90;>A3882FF@7D=EA;,"9114,8900,7709,7439,6783,6281,5580,5562,7117,5...","c99bf8d8-0be9-4ba9-8c41-4c49ba953f39,40364381-...","0,0,16,16,0,0,0,0,0,0,16,16,0,0,0,0,16,16,0,0,...",XXY[TZRYVWW[XYWZT]ZYYZRX[]Y[]]ZT[[<]V]XVWY[XS,52,",,,...,,.$,....,,...,,.,..,,.,..,..,.,,,..,,.....",38>A479:B7B<44G2@CA+B+475>298@=75D>1<?;3(9@5=?...,...,2,2,"..*,......,,....,,........,.,.,.,,,,,.,,....,",",,,...,,.$,....,,...,,.,..,,.,..,..,.,,,..,,.....",",,,...,,.$,....,,...,,.,..,,.,..,..,.,,,..,,.....",[],[],"..*,......,,....,,........,.,.,.,,,,,.,,....,",[],[]
76142565,45,"..,,.....,,....,,T.......t.,.,.,,,,,.,,....,^Z,","<*;6,'&&137+916?A'0'9)C*/%73'8875;4/>.5'<+*16","9178,8968,7762,7503,6844,5646,5627,7183,5344,5...","c99bf8d8-0be9-4ba9-8c41-4c49ba953f39,40364381-...","0,0,16,16,0,0,0,0,0,16,16,0,0,0,0,16,16,0,0,0,...",XXY[TRYVWW[XYWZT]ZYYZRX[]Y[]]ZT[[<]V]XVWY[XSZ,51,",,,...,,,....,,..G,,.,-1g..,,.,..,..G,,,..,,.....","764J0159<A)./=7(%(;.'$#$%6&)72:.5$'7=(/<7,30./...",...,2,2,"..,,.....,,....,,T.......t.,.,.,,,,,.,,....,,",",,,...,,,....,,..G,,.,-1g..,,.,..,..G,,,..,,.....",",,,...,,,....,,..G,,.,..,,.,..,..G,,,..,,........",[],[g],"..,,.....,,....,,T.......t.,.,.,,,,,.,,....,,",[],[]
76142572,46,".-1C.,,.-1C...-1C.,,....-1C,,.......-1C.,.,.,+...","@?/%;957A,;B=1>46>=@<'C??$F60,@1)9&&<%)>;=1&74","9185,8975,7769,7510,6851,5653,5634,7190,5351,5...","c99bf8d8-0be9-4ba9-8c41-4c49ba953f39,40364381-...","0,0,16,16,0,0,0,0,0,16,16,0,0,0,0,16,16,0,0,0,...",XXY[TRYVWW[XYWZT]ZYYZRX[]Y[]]ZT[[<]V]XVWY[XSZ[,51,",,,..-1C.c,,...-1C.,,...,,.,..,,.,..,...,*,.+1...",?&+=>:+$'@:594=>=.(38$93-$>(A7-:=<$(?9@$.>@<:D...,...,2,2,".-1C.,,.-1C...-1C.,,....-1C,,.......-1C.,.,.,+...",",,,..-1C.c,,...-1C.,,...,,.,..,,.,..,...,*,.+1...",",,,...c,,....,,...,,.,..,,.,..,...,*,..,,........",[C],"[C, C, C, C, C]","..,,.....,,....,,........,.,.,.,*,**.,*....,,,",[c],"[C, C, C, C, C, C, c]"
76142791,46,"..,,....,,....,,........,.,.,.,,,,,.,,....,c,^W.",=<89/7<;7:433;7@:7:<5=6@@84(;8?::;;7<399=;;#:8,"9399,9189,7989,7722,7057,5850,7402,5563,5476,5...","c99bf8d8-0be9-4ba9-8c41-4c49ba953f39,40364381-...","0,0,16,16,0,0,0,0,16,16,0,0,0,0,16,16,0,0,0,0,...",XXY[TYVWW[XYWZT]ZYYZRX[]Y[]]ZT[[<]V]XVWY[XSZ[W,50,",,...,,-2at,...*,,...,,.,..-3ATG,,.,..,...,,,....",":6,&=?)>>;<(95<0<89:=?&29;<7;<@:5+;996?86<=49?...",...,2,2,"..,,....,,....,,........,.,.,.,,,,,.,,....,c,.",",,...,,-2at,...*,,...,,.,..-3ATG,,.,..,...,,,....",",,...,,,...*,,...,,.,..,,.,..,...,,,..,,.....,...",[],"[at, ATG]","..,,....,,....,,........,.,.,.,,,,,.,,....,c,.",[],[]
76142869,47,".*,,-1g..G.,,....,,........,.,.,.,,,,,-1g.,,.....","'&;92;*.?*,1,01@)944),8;-3($85(67:&>57$41;+;932","9475,9265,8066,7803,7136,5928,7478,5635,5555,5...","c99bf8d8-0be9-4ba9-8c41-4c49ba953f39,40364381-...","0,0,16,16,0,0,0,0,16,16,0,0,0,0,16,16,0,0,0,0,...",XXY[TYVWW[XYWZT]ZYYZRX[]Y[]]ZT[[<]V]XVWY[XSZ[WU,51,",,...,,,G...,,...,*T,G.,,.,..t...g,,..,,....-1...","*99+.)?9/&),<)20&20(+$).'1<=-170/%:7+(%/-*.%<-...",...,2,2,".*,,-1g..G.,,....,,........,.,.,.,,,,,-1g.,,.....",",,...,,,G...,,...,*T,G.,,.,..t...g,,..,,....-1...",",,...,,,G...,,...,*T,G.,,.,..t...g,,..,,.....g...",[],[G],".*,,..G.,,....,,........,.,.,.,,,,,.,,....,,,..",[],"[g, g]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76239175,45,",...,.,,,,,,,,A.,.,.,,.+2AA,,A,,....,..,..,,,....",">5=5=-5%/67@;6,9B3=:=9'A)%>?+'.$=5<=A-<:?,&2)","10933,10525,8979,8617,7688,7588,6637,6588,6117...","d4e6733f-0b2e-4dae-b6bb-50906aae9d3f,c6084eeb-...","16,0,0,0,16,0,16,16,16,16,16,16,16,16,0,0,2064...",VU[TW]UY[TYZU[]]WQZNZ[[[]]ZT[]QZ]]]Z[YXWZ]P]W,52,"..,,,,.*,,..,.,,,,,..,,....,..,..,,,,+1t.,..C....","';9BB?6$=..-.3:-6?./A<?&()6/9.==+<(&%$=,/#<;F1...",...,2,2,

In [ ]:
data = pd.read_csv(
    path,
    dtype={
        DataSchema.AMOUNT: float,
        DataSchema.CATEGORY: str,
        DataSchema.DATE: str,
    },
    parse_dates=[DataSchema.DATE],
)

In [40]:
import pandas as pd
import numpy as np
import locale
import random, string

N = 1000000
np.random.seed(0)
rand_name_lst=[''.join(random.choices(string.ascii_uppercase + string.digits, k=5)) \
                            for i in range(N)]
rand_int_lst = np.random.randint(912131,18293293,size=N)
df = pd.DataFrame({'name': rand_name_lst , 'size': rand_int_lst})

df.sample(5)

def convert_size(x):
    kb = round((x / 1024.0),2)
    mb = round((x / (1024.0 ** 2)),2)
    gb = round((x / (1024 **3)),2)

    return pd.Series([kb, mb, gb])
df[['size_kb','size_mb','size_gb']]=df['size'].apply(convert_size)
df


,name,size,size_kb,size_mb,size_gb
0,EBBZP,9237935,9021.42,8.81,0.01
1,A7HC2,3127235,3053.94,2.98,0.00
2,UTJ34,6069830,5927.57,5.79,0.01
3,CTLJ9,15951978,15578.10,15.21,0.01
4,CELYH,8556300,8355.76,8.16,0.01
...,...,...,...,...,...
999995,5UM50,10262913,10022.38,9.79,0.01
999996,FFHPH,11270724,11006.57,10.75,0.01
999997,3WJH2,13253839,12943.20,12.64,0.01
999998,89ZJE,16825160,16430.82,16.05,0.02


In [41]:
df['size_kb'],df['size_mb'],df['size_gb']=convert_size(df['size'])
df

,name,size,size_kb,size_mb,size_gb
0,EBBZP,9237935,9021.42,8.81,0.01
1,A7HC2,3127235,3053.94,2.98,0.00
2,UTJ34,6069830,5927.57,5.79,0.01
3,CTLJ9,15951978,15578.10,15.21,0.01
4,CELYH,8556300,8355.76,8.16,0.01
...,...,...,...,...,...
999995,5UM50,10262913,10022.38,9.79,0.01
999996,FFHPH,11270724,11006.57,10.75,0.01
999997,3WJH2,13253839,12943.20,12.64,0.01
999998,89ZJE,16825160,16430.82,16.05,0.02


In [25]:
import re

txt = ".-1C.,,.-1C...-1C.,,....-1C,,.."
re.findall("\+[0-9]+[ACGTNacgtn]+", txt), re.findall('\-[0-9]+[ACGTNacgtn]+', txt)

([], ['-1C', '-1C', '-1C', '-1C'])

In [26]:
from collections import Counter




In [27]:
for indel in cc.keys():
    txt = ''.join(txt.split(indel))
txt

'..,,.....,,....,,..'

In [4]:
s1 ='CPPffff'
s2='gggg'

any(map(s1.startswith, ['CPP', 'CGD', 'CPG', 'CNE', 'CNL']))

True

In [11]:
help(df.plot)

Help on PlotAccessor in module pandas.plotting._core object:

class PlotAccessor(pandas.core.base.PandasObject)
 |  PlotAccessor(data)
 |  
 |  Make plots of Series or DataFrame.
 |  
 |  Uses the backend specified by the
 |  option ``plotting.backend``. By default, matplotlib is used.
 |  
 |  Parameters
 |  ----------
 |  data : Series or DataFrame
 |      The object for which the method is called.
 |  x : label or position, default None
 |      Only used if data is a DataFrame.
 |  y : label, position or list of label, positions, default None
 |      Allows plotting of one column versus another. Only used if data is a
 |      DataFrame.
 |  kind : str
 |      The kind of plot to produce:
 |  
 |      - 'line' : line plot (default)
 |      - 'bar' : vertical bar plot
 |      - 'barh' : horizontal bar plot
 |      - 'hist' : histogram
 |      - 'box' : boxplot
 |      - 'kde' : Kernel Density Estimation plot
 |      - 'density' : same as 'kde'
 |      - 'area' : area plot
 |      - 'p

,a
a,0
b,9


In [6]:
%time
sample_id = 'sample_id'
p='/Users/liam/Downloads/functional/'
gzs =glob(p+'BBB*.MICROBA.sample.tsv.gz')
print(len(gzs))

for i in gzs:

    df = pd.read_csv(i, index_col =0, sep='\t')
    assert len(df.columns) == 1
    df.columns = [sample_id]

    print(len(df[sample_id]))


CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 2.15 µs
28
553942
387157
506655
522019
436605
503589
418377
570555
641810
633009
498685
530719
468912
627153
487607
607455
632898
614981
432772
489456
572237
716704
508945
378263
410248
587297
510855
597782


In [8]:
df = pd.read_csv('/Users/liam/Downloads/BBB6542-PBC4-LBC2-R006-FSS.MICROBA.sample.tsv.gz', index_col =0, sep='\t')
df

,Count
ID,
MIG20000000028,1
MIG20000000056,4
MIG20000000075,1
MIG20000000096,2
MIG20000000124,3
...,...
MIG20073909726,1
MIG20073909781,10
MIG20073909783,1


In [2]:
@dataclass
class C2:
    x: int
    def f(self, y):
        print(self.x*y)

@dataclass
class C1(C2):
    def f(self, y):
        print(self.x+y, 'fu')

c1 = C1(1)
c1.f(2)

3 fu


In [3]:
d={'a': C2}
c = d.get('a', C1)(1)
c.f(2)

2


In [4]:
CPY_BACKGROUND_SPECIES = "Taxon\n__Pseudomonas aeruginosa\ns__Escherichia coli\ns__Salmonella enterica\ns__Lactobacillus_H fermentum\ns__Enterococcus faecalis\ns__Staphylococcus aureus\ns__Listeria monocytogenes_A\ns__Listeria monocytogenes_B\ns__Listeria monocytogenes\ns__Bacillus subtilis_D\ns__Saccharomyces group A\ns__Cryptococcus neoformans\ns__Bacillus marinus\ns__Escherichia flexneri"
CPD_BACKGROUND_SPECIES = "Taxon\ns__Acinetobacter baumannii\ns__Actinomyces_B odontolyticus\ns__Bacillus_A cereus_AD\ns__Bacteroides_B vulgatus\ns__Bifidobacterium adolescentis\ns__Clostridium beijerinckii\ns__Cutibacterium acnes\ns__Deinococcus radiodurans\ns__Enterococcus faecalis\ns__Escherichia coli\ns__Helicobacter pylori_AS\ns__Lactobacillus gasseri\ns__Neisseria meningitidis\ns__Porphyromonas gingivalis\ns__Pseudomonas aeruginosa_A\ns__Staphylococcus aureus\ns__Staphylococcus epidermidis\ns__Streptococcus agalactiae\ns__Streptococcus mutans\ns__Helicobacter pylori\ns__Escherichia flexneri\ns__Pauljensenia odontolyticus_A"
CPD_BACKGROUND_SPECIES2 = "Taxon\ns__Acinetobacter ff\ns__Actinomyces_B ff\ns__Bacillus_A ff\ns__Bacteroides_B vulgatus\ns__Bifidobacterium adolescentis\ns__Clostridium beijerinckii\ns__Cutibacterium acnes\ns__Deinococcus radiodurans\ns__Enterococcus faecalis\ns__Escherichia coli\ns__Helicobacter pylori_AS\ns__Lactobacillus gasseri\ns__Neisseria meningitidis\ns__Porphyromonas gingivalis\ns__Pseudomonas aeruginosa_A\ns__Staphylococcus aureus\ns__Staphylococcus epidermidis\ns__Streptococcus agalactiae\ns__Streptococcus mutans\ns__Helicobacter pylori\ns__Escherichia flexneri\ns__Pauljensenia odontolyticus_A"


In [5]:
df =pd.DataFrame([CPY_BACKGROUND_SPECIES.split('\n'), CPD_BACKGROUND_SPECIES.split('\n'), CPD_BACKGROUND_SPECIES2.split('\n')]).T
df.columns = ['a', 'b', 'b2']
df.head()



,a,b,b2
0,Taxon,Taxon,Taxon
1,__Pseudomonas aeruginosa,s__Acinetobacter baumannii,s__Acinetobacter ff
2,s__Escherichia coli,s__Actinomyces_B odontolyticus,s__Actinomyces_B ff
3,s__Salmonella enterica,s__Bacillus_A cereus_AD,s__Bacillus_A ff
4,s__Lactobacillus_H fermentum,s__Bacteroides_B vulgatus,s__Bacteroides_B vulgatus


In [6]:
df["exists"] = df.drop("b2", 1).isin(df["b2"]).any(1)
df

/var/folders/5k/61vjt_z14fg3thmw_wts19f40000gp/T/ipykernel_41367/2934274232.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df["exists"] = df.drop("b2", 1).isin(df["b2"]).any(1)


,a,b,b2,exists
0,Taxon,Taxon,Taxon,True
1,__Pseudomonas aeruginosa,s__Acinetobacter baumannii,s__Acinetobacter ff,False
2,s__Escherichia coli,s__Actinomyces_B odontolyticus,s__Actinomyces_B ff,False
3,s__Salmonella enterica,s__Bacillus_A cereus_AD,s__Bacillus_A ff,False
4,s__Lactobacillus_H fermentum,s__Bacteroides_B vulgatus,s__Bacteroides_B vulgatus,True
5,s__Enterococcus faecalis,s__Bifidobacterium adolescentis,s__Bifidobacterium adolescentis,True
6,s__Staphylococcus aureus,s__Clostridium beijerinckii,s__Clostridium beijerinckii,True
7,s__Listeria monocytogenes_A,s__Cutibacterium acnes,s__Cutibacterium acnes,True
8,s__Listeria monocytogenes_B,s__Deinococcus radiodurans,s__Deinococcus radiodurans,True
9,s__Listeria monocytogenes,s__Enterococcus faecalis,s__Enterococcus faecalis,True


In [7]:
df = pd.read_csv('/Users/liam/Downloads/all_trusted_flex_v2_r322.txt', header=None, names=['controls'])
df[['name', 'foo', 'bar', 'run', 'CTL']] = df.controls.str.split('-', expand=True)
df
CTLs=set(df.CTL)
if len(CTLs) != 1 and CTLs.pop() != 'CTL':
    raise ValueError(f'Type should be CTL, only; {CTLs}')

df['control_type'] = df.name.str[:3]
df=df[['controls', 'name', 'run', 'control_type']]
df.value_counts('control_type')
df.sort_values('run', inplace=True)
df.tail(50).controls.to_csv('~/Downloads/hhhh.lof', index=False, header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/liam/Downloads/all_trusted_flex_v2_r322.txt'

In [ ]:
df = pd.read_csv('/Users/liam/Downloads/R388/CPG/sp_test/profiles_counts_background/profile_summary_stats.tsv', sep='\t')
df.Taxon

0            s__Acetatifactor sp900066565
1          s__Adlercreutzia equolifaciens
2                 s__Agathobacter rectale
3      s__Agathobaculum butyriciproducens
4            s__Akkermansia muciniphila_B
                      ...                
112                s__UBA1417 sp003531055
113                    s__UBA3818 MIC8425
114                    s__UBA7160 MIC6745
115                    s__UBA7160 MIC9207
116                    s__UBA7182 MIC8422
Name: Taxon, Length: 117, dtype: object

In [ ]:
dfh= pd.read_csv('/Users/liam/Downloads/PRJEB38148_human.csv', index_col=0)
print(dfh.shape)

dfh.head(2)

(369, 42)


,Assay Type,AvgSpotLen,Bases,bio_material,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,SRA Study,Submitter_Id,diet,days_post_colonization,donor_microbiota,experimental_group,Isolator,Mouse_ID,Collection_day,Participant_ID
Run,,,,,,,,,,,,,,,,,,,,,
ERR5261198,OTHER,269,7990595975,Stool,PRJEB38148,SAMEA7772329,2814020302,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"fastq,sra",...,ERP121534,PF_S.13_Day19,Pea fiber (2 snacks a day),NaN,NaN,NaN,NaN,NaN,19.0,S.13
ERR5261199,OTHER,265,6859111259,Stool,PRJEB38148,SAMEA7772328,2276073800,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"fastq,sra",...,ERP121534,PF_S.13_Day18,Pea fiber (2 snacks a day),NaN,NaN,NaN,NaN,NaN,18.0,S.13


In [ ]:
dfjoel= pd.read_csv('/Users/liam/Downloads/PRJEB38148.metadata.tsv', sep='\t', index_col=3, header=None)
print(dfjoel.shape)

dfjoel.head(2)

(363, 6)


,0,1,2,4,5,6
3,,,,,,
ERR5349076,PRJEB38148,SAMEA7772053,ERX5137452,HS2,2Fib4Fib_S.09,Day11
ERR5349077,PRJEB38148,SAMEA7772052,ERX5137453,HS2,2Fib4Fib_S.09,Day10


In [ ]:
df = dfh.join(dfjoel, how='inner')
print(df.shape)
df.head()

(363, 48)


,Assay Type,AvgSpotLen,Bases,bio_material,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Isolator,Mouse_ID,Collection_day,Participant_ID,0,1,2,4,5,6
ERR5261198,OTHER,269,7990595975,Stool,PRJEB38148,SAMEA7772329,2814020302,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"fastq,sra",...,NaN,NaN,19.0,S.13,PRJEB38148,SAMEA7772329,ERX5051565,HS1,PF_S.13,Day19
ERR5261199,OTHER,265,6859111259,Stool,PRJEB38148,SAMEA7772328,2276073800,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"fastq,sra",...,NaN,NaN,18.0,S.13,PRJEB38148,SAMEA7772328,ERX5051566,HS1,PF_S.13,Day18
ERR5261201,OTHER,264,6016100973,Stool,PRJEB38148,SAMEA7772330,2001778622,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"sra,fastq",...,NaN,NaN,20.0,S.13,PRJEB38148,SAMEA7772330,ERX5051568,HS1,PF_S.13,Day20
ERR5261202,OTHER,264,5721647335,Stool,PRJEB38148,SAMEA7772334,1886994029,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"sra,fastq",...,NaN,NaN,36.0,S.13,PRJEB38148,SAMEA7772334,ERX5051569,HS1,PF_S.13,Day36
ERR5261204,OTHER,268,6275097722,Stool,PRJEB38148,SAMEA7772335,2100540392,WASHINGTON UNIVERSITY CENTER FOR GENOME SCIENCES,public,"fastq,sra",...,NaN,NaN,39.0,S.13,PRJEB38148,SAMEA7772335,ERX5051571,HS1,PF_S.13,Day39


In [ ]:
df = pd.DataFrame(list())
for sample_id, mac_summary in classify_files.items():
    try:
        summ_df = pd.read_csv(mac_summary, sep="\t")
        if 'None' in summ_df.columns:
            self.log.warn(f'No MGDB-ANI-CLASSIFY summary found for sample {sample_id}. Marking as None.')
            df.loc[sample_id, 'Query species assignment'] = 'None'
            df.loc[sample_id, 'Query taxonomy'] = 'None'
            df.loc[sample_id, 'Closest representative'] = 'None'
            df.loc[sample_id, 'ANI'] = 'None'
            df.loc[sample_id, 'AF'] = 'None'
            df.loc[sample_id, 'Representative taxonomy'] = 'None'

        else:
            df.loc[sample_id, 'Query species assignment'] = summ_df.iloc[0]['Query species assignment']
            df.loc[sample_id, 'Query taxonomy'] = summ_df.iloc[0]['Query taxonomy']
            df.loc[sample_id, 'Closest representative'] = summ_df.iloc[0]['Closest representative']
            df.loc[sample_id, 'ANI'] = summ_df.iloc[0]['ANI']
            df.loc[sample_id, 'AF'] = summ_df.iloc[0]['AF']
            df.loc[sample_id, 'Representative taxonomy'] = \
                summ_df.iloc[0]['Representative taxonomy']
    except Exception:
        self.log.error(f"MGDB-ANI-CLASSIFY results are incomplete. Review file {mac_summary}")
        raise
df.index.name = 'Sample'


In [ ]:
series = []

summ_df = pd.read_csv('/Users/liam/Downloads/testa/BBL6672-PFG3-LFG2-R385-BUG.sp_classification.tsv', sep="\t", index_col=0)
summ_df

,Query species assignment,Query taxonomy,Closest representative,ANI,AF,Representative taxonomy
Query ID,,,,,,
BBL6672-PFG3-LFG2-R385-BUG,s__Bifidobacterium adolescentis,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,G000010425,97.8854,0.857759,d__Bacteria;p__Actinobacteriota;c__Actinobacte...


In [ ]:
sample_id='gg'
new_df = []
sample_IDs=['BBL6672-PFG3-LFG2-R385-BUG', 'BBL6672-PFG3-LFG2-R385-BUG2']
cols = ['Query species assignment', 'Query taxonomy', 'Closest representative', 'ANI', 'AF', 'Representative taxonomy']
for sample_ID in sample_IDs:
    summ_df = pd.read_csv(f'/Users/liam/Downloads/testa/{sample_ID}.sp_classification.tsv', sep="\t", index_col=0)
    if 'None' in summ_df.columns:
        #print(111,pd.Series({k: 'None' for k in cols}, name=sample_ID), '\n')
        new_df.append(pd.Series({k: 'None' for k in cols}, name=sample_ID))
    else:
        #print(2222,summ_df.T[sample_ID])
        new_df.append(summ_df.T[sample_ID])
df = pd.DataFrame(new_df)
df.index.name = 'Sample'
df

,Query species assignment,Query taxonomy,Closest representative,ANI,AF,Representative taxonomy
Sample,,,,,,
BBL6672-PFG3-LFG2-R385-BUG,s__Bifidobacterium adolescentis,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,G000010425,97.8854,0.857759,d__Bacteria;p__Actinobacteriota;c__Actinobacte...
BBL6672-PFG3-LFG2-R385-BUG2,None,None,None,None,None,None


In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}

ser = pd.Series({'a': 1, 'b': 2, 'c': 3}, name='gg')

ser

a    1
b    2
c    3
Name: gg, dtype: int64

In [ ]:
p='/Users/liam/Downloads/testa/'
samples= ['AssemblySummary1','AssemblySummary2']
df = pd.DataFrame(list())
for sample_id in samples:
    with open(p+sample_id+'.txt') as f:
        for line in f:
            line_split = line.split()
            if len(line_split) == 0:
                continue
            if line_split[0] == '#':
                continue
            if line_split[0].lower() == 'contamination':
                df.loc[sample_id, 'Contamination'] = line_split[-1]
            elif line_split[0].lower() == 'completeness':
                df.loc[sample_id, 'Completeness'] = line_split[-1]
            elif line_split[0].upper() == 'N50':
                if sample_id in df.index:
                    if pd.isnull(df.loc[sample_id, 'N50']):
                        df.loc[sample_id, 'N50'] = line_split[-1]
                else:
                    df.loc[sample_id, 'N50'] = line_split[-1]
df.index.name = 'Sample'
df


,N50,Completeness,Contamination
Sample,,,
AssemblySummary1,1389366,100.00,0.00
AssemblySummary2,1399366,90.00,0.00


In [ ]:
d = defaultdict(dict)
col_names = ['contamination', 'completeness', 'N50']
p='/Users/liam/Downloads/testa/'
samples= ['AssemblySummary1','AssemblySummary2']
for sample_id in samples:
    with open(p+sample_id+'.txt') as f:
        for line in f:
            bits = line.strip().split()
            if len(bits) == 2 and bits[0] in col_names:
                d[sample_id].setdefault(bits[0].capitalize(), bits[1])
                print(sample_id, d)
df = pd.DataFrame.from_dict(d, orient='index')
df.index.name = 'Sample'

df

AssemblySummary1 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366'}})
AssemblySummary1 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366'}})
AssemblySummary1 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366'}})
AssemblySummary1 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366', 'Completeness': '100.00'}})
AssemblySummary1 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366', 'Completeness': '100.00', 'Contamination': '0.00'}})
AssemblySummary2 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366', 'Completeness': '100.00', 'Contamination': '0.00'}, 'AssemblySummary2': {'N50': '1399366111'}})
AssemblySummary2 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366', 'Completeness': '100.00', 'Contamination': '0.00'}, 'AssemblySummary2': {'N50': '1399366111'}})
AssemblySummary2 defaultdict(<class 'dict'>, {'AssemblySummary1': {'N50': '1389366', 'Completeness': '100.00', 'Contamination'

,N50,Completeness,Contamination
Sample,,,
AssemblySummary1,1389366,100.00,0.00
AssemblySummary2,1399366111,90.00,0.00


In [ ]:
p='/Users/liam/Downloads/testa/'
samples= ['AssemblySummary1','AssemblySummary2']
col_names = ['contamination', 'completeness', 'N50']
df = pd.concat([
        pd.read_csv(
                p+sample_id+'.txt',
                sep='\t',
                skip_blank_lines=True,
                index_col=None,
                names=['foo', sample_id])
        .query(f'foo in {col_names}')
        .reset_index(drop=True)
        .drop([1,2])
        .set_index('foo') 
        .T
        for sample_id in samples
        ])
df.columns = map(str.capitalize, col_names)
df.index.name = 'Sample'

df

,Contamination,Completeness,N50
Sample,,,
AssemblySummary1,1389366,100.00,0.00
AssemblySummary2,1399366,90.00,0.00


In [ ]:
df = pd.read_csv(
    p+sample_id+'.txt',
    sep='\t',
    skip_blank_lines=True,
    index_col=None,
    names=['a', sample_id]) \
    .query(f'a in {col_names}') \
    .reset_index(drop=True) \
    .drop([1,2]) \
    .set_index('a') \
    .T
#df.index = ['sa']
df

a,N50,completeness,contamination
AssemblySummary2,1399366,90.00,0.00


In [ ]:

df = pd.concat([
    pd.read_csv(assembly_summary, comment='#', sep=' ') for sample_id, assembly_summary in assembly_files.items()
])

In [ ]:
with open('/Users/liam/Downloads/r394_release_IDs.txt') as fin:
    ids = []
    for line in fin:
        ids.append(line.strip())

In [ ]:
with open('/Users/liam/Downloads/r394_all_ids.txt') as fin, open('/Users/liam/Downloads/r394_fin.txt', 'w') as fout:
    for line in fin:
        if line.strip().split('_')[0] in ids:
            fout.write(line)

In [ ]:
with open('/Users/liam/Downloads/MBS-1342_attachments/R396.commands_PST.sh', 'r') as fin:
    ids103=[]
    bb_ids103=[]
    for line in fin:
        ids103.append(line.strip().split()[7])
        bb_ids103.append(line.strip().split()[7].split('-')[0])
len(ids103), ids103[:4], bb_ids103[:4]

(103,
 ['BBC7235-PFH0-LFH0-R396-FSS',
  'BBC9835-PFH0-LFH0-R396-FSS',
  'BBD1787-PFH0-LFH0-R396-FSS',
  'BBF2314-PFH0-LFH0-R396-FSS'],
 ['BBC7235', 'BBC9835', 'BBD1787', 'BBF2314'])

In [ ]:
with open('/Users/liam/Downloads/ids_94.txt', 'r') as fin:
    ids94=[]
    for line in fin:
        ids94.append(line.strip())
len(ids)

94

In [ ]:
set(ids94).difference(set(bb_ids103))

set()

In [ ]:
set(bb_ids103).difference(set(ids94))

{'BBJ7165',
 'BBJ7283',
 'BBK9725',
 'BBK9727',
 'BBK9732',
 'BBK9737',
 'BBK9756',
 'BBK9775',
 'BBK9816'}

In [ ]:
def a():
    return 1,

x, = a()
x

1

In [ ]:
from enum import Enum
class Color(Enum):
    ['ff'] = 1
    GREEN = 2
    BLUE = 3

Color(1)

<Color.RED: 1>

In [ ]:
from enum import Enum


class Season(Enum):
    SPRING = 1
    SUMMER = 2
    AUTUMN = 3
    WINTER = 4


seas = Season.SPRING
print(111,seas)

if seas == Season.SPRING:
    print("Spring")

print(list(Season))

111 Season.SPRING
Spring
[<Season.SPRING: 1>, <Season.SUMMER: 2>, <Season.AUTUMN: 3>, <Season.WINTER: 4>]


In [ ]:
from enum import Enum
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

In [ ]:
a = Color['RED']
a.name + 'a'

'REDa'